# Synopsis

This notebook will explain the following topics and concepts:

**Concatenation**
- Rows and Columns

**Merging Data**
- left
- right
- inner
- outer

**Joining Data**
- left
- right
- inner
- outer

**Grouping Data**
- by time
- by columns

# Importing libraries & Load Data

We'll use the same csv files as we used in chapter 3.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


# format for floats
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
df_GOOGL = pd.read_csv(filepath_or_buffer='../Data/GOOGL.csv', index_col='Date', parse_dates=True)
df_IBM = pd.read_csv(filepath_or_buffer='../Data/IBM.csv', index_col='Date', parse_dates=True)
df_MSFT = pd.read_csv(filepath_or_buffer='../Data/MSFT.csv', index_col='Date', parse_dates=True)

# Concatenation

- Glues together DataFrames, without much intelligence.
- Dimensions should match along the axis you are concatenating on. 
- Use **pd.concat** and pass in a list of DataFrames to concatenate together

## Create a few simple DataFrames 

In [ ]:
date_range = pd.date_range(start='2017', freq='BQ', periods=4)
cols  = ['Open', 'Close']

df1 = df_IBM.reindex(date_range)[cols]
df2 = df_GOOGL.reindex(date_range)[cols]
df3 = df_MSFT.reindex(date_range)[cols]

## Concatenate

The default is to concatenate the rows

In [ ]:
df_all = pd.concat([df1, df2, df3])
df_all

## Concatenate the columns
Pass the `axis = 1` parameter to pd.concat

In [ ]:
df_all = pd.concat([df1, df2, df3], axis=1)
df_all


# Merging

Pandas has two important functions for joining DataFrames together which intelligently try to align values from selected columns of each DataFrame. These functions are called **merge** and **join**. These functions use a similar logic to joins in SQL.

First we will look at merge.

**There are 4 Different types of merge**
- **Inner Merge** – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right DataFrames.
- **Left Merge** – (aka left merge or left join) Keep every row in the left DataFrame. Where there are missing values of the “on” variable in the right DataFrame, add empty / NaN values in the result.
- **Right Merge** – (aka right merge or right join) Keep every row in the right DataFrame. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.
- **Outer Merge** – A full outer join returns all the rows from the left DataFrame, all the rows from the right DataFrame, and matches up rows where possible, with NaNs elsewhere.

## Create some sample DataFrames

Just a few days worth of Data from Google and IBM

Note the difference in date ranges

In [ ]:
cols = ['High', 'Low']

df1 = df_IBM[cols]['2017-Jan-01':'2017-Jan-06'].sort_index()
df2 = df_GOOGL[cols]['2017-Jan-05':'2017-Jan-10'].sort_index()

# show both dataframes
print("== IBM ==")
display(df1)
print("== GOOGLE ==")
display(df2)

## Inner Merge

Only keep values for Dates found in both left (df1) and right (df2)

In [ ]:
# This version of merge works in latest Pandas
pd.merge(df1, df2, how='inner', on='Date')

# This version of merge works in older Pandas
pd.merge(df1, df2, how='inner', left_index=True, right_index=True)


## Left Merge

- Keep everything in the left DataFrame.
- Where nothing exists in the right DataFrame, fill with NaN ("Not a Number" - these are empty values).
- Use the suffixes parameter to override the x_ and y_ defaults

In [ ]:
srcs = ['_IBM','_GOOGL']

pd.merge(df1, df2, how='left', on='Date', suffixes=srcs)

pd.merge(df1, df2, how='left', left_index=True, right_index=True,  suffixes=srcs)

## Right Merge

- Keep everything in the right DataFrame
- Where nothing exists in the left DataFrame, fill with NaN

In [ ]:
pd.merge(df1, df2, how='right', on='Date', suffixes=srcs)

pd.merge(df1, df2, how='right', left_index=True, right_index=True, suffixes=srcs)

## Outer Merge

Keep everything in both left and right DataFrames, fill with NaN where no data present

In [ ]:
pd.merge(df1, df2, how='outer', on='Date', suffixes=srcs)



# Joining

- The second pandas function for intelligently combining DataFrames is called **join**.
- Join is **very** similar to merge.
- As with merge, the **how** parameter takes inner, outer, left or right.
- As with merge, the **on** parameter is the name of a column to join on.
- However there is one major difference:
  - When using join the "on" **must** be the index in at least one of the DataFrames.
  - Merge will allow the "on" to be a regular column in **both** DataFrames.

The syntax for calling the two functions is also slightly different:
- **join**  : df1.join(df2, how="inner", on="Date")
- **merge** : pd.merge(df1, df2, how="inner", on="Date")



In [ ]:
cols1 = ['High', 'Low']
cols2 = ['Open', 'Close']
df1 = df_IBM[cols1]['2017-Jan-01':'2017-Jan-07'].sort_index()
df2 = df_IBM[cols2]['2017-Jan-04':'2017-Jan-11'].sort_index()

# show both dataframes
print("== High & Low ==")
display(df1)
print("== Open & Close ==")
display(df2)



In [ ]:
# If we don't specify an "on" then we will join "on" the index of both DataFrames
df1.join(df2)


In [ ]:
df1.join(df2, how='right')


In [ ]:
df1.join(df2, how='inner')


In [ ]:
df1.join(df2, how='outer')


# Grouping Data

Pandas provides functions that allow us to group rows of data together and call aggregate functions on them as a unit e.g. mean, max, min, std, etc.

To create a group we call the **groupby** method on a DataFrame.

e.g. Create groups where the 'Industry' column is the same:
    - df1.groupby('Industry')

## Group by Columns

Use the **by** parameter and supply a column name

In [ ]:
df = pd.read_csv(filepath_or_buffer='../Data/groups.csv',index_col='Date', parse_dates=True)

In [ ]:
# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Sector').mean(numeric_only=True)

# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Rep').mean(numeric_only=True)

# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Portfolio').mean(numeric_only=True)

# Or we could create a variable to store the name of the function
func = 'std'
df.groupby(by='Sector').agg(func,numeric_only=True)



## Group by Time Period
Often we want to group according to a frequency e.g. a group of all values in a single business quarter.

We can then call mean for all of the groups, e.g. get the mean Volume per business quarter.

A convenient way to group by a frequency is to use the Grouper object (in the pandas namespace).

- pd.Grouper
- Most commonly used to pass in a frequency
- Group by frequencies: B (business day), BQ (business quarter), M (month), Y (year), etc.
- It's also possible to group by specific time frequencies e.g. 15d, 1h30min etc.
  - See the list of frequency aliases: http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
df.dtypes

In [ ]:
df.groupby(pd.Grouper(freq='BQ')).mean(numeric_only=True)

# Or we could create a variable to store the grouper
by_BMonth = pd.Grouper(freq='BM')
df.groupby(by=by_BMonth).mean(numeric_only=True)

# Or use teh agg and a list of functions
funcs = 'mean'
df.groupby(by=by_BMonth).agg(funcs, numeric_only=True)


# Finally we can supply a list of groupers
by_BMonth = pd.Grouper(freq='BM')
by_Rep = pd.Grouper(key='Rep')
by_Sector = pd.Grouper(key='Sector')

# And perform a variety of slice and dice operations
groups1 = [by_Rep, by_BMonth, by_Sector]
df.groupby(by=groups1).agg(funcs, numeric_only=True)

groups2 = [by_Sector, by_Rep, by_BMonth]
df.groupby(by=groups2).agg(funcs, numeric_only=True)

groups3 = [by_BMonth, by_Sector, by_Rep]
df.groupby(by=groups3).agg(funcs, numeric_only=True)